# Calculation Functions Figure 3 

In [ ]:
# Creates labels for y ticks 
def yTicks(y): 
    labels = []

    maxY = max(y)
    intMaxY = math.ceil(float(maxY))
            
    ticks = [x for x in range(-abs(intMaxY), (intMaxY + 1))]
    
    for num in ticks:
        if num == 0: labels.append('no change')
        elif num < 0: 
            modNum = 2**abs(num)
            label = f'1/{modNum}x'
            labels.append(label)
        else:
            modNum = 2**num
            label = f'{modNum}x'
            labels.append(label)
    
    return(ticks, labels)

In [ ]:
def differentialExpScatter(sampleObj, sampleName, divideby, xFilePath, yFilePath):
    sc.tl.rank_genes_groups(sampleObj, divideby, method='wilcoxon')
    
    score_df = pd.DataFrame(sampleObj.uns['rank_genes_groups']['scores'])
    geneName_df = pd.DataFrame(sampleObj.uns['rank_genes_groups']['names'])

    geneName_lst = list(sampleObj.var_names)

    samples = list(np.unique(sampleObj.obs[divideby]))
    sample1_name = samples[0]
    sample2_name = samples[1]
    nameLst = [sample1_name, sample2_name]

    sample1_obj = sampleObj[sampleObj.obs[divideby].isin([sample1_name]),:]
    sample2_obj = sampleObj[sampleObj.obs[divideby].isin([sample2_name]),:]

    sample1_geneExpression = sample1_obj.X.toarray()
    sample1_meanGeneExpression = list(np.mean(sample1_geneExpression, axis=0))

    sample2_geneExpression = sample2_obj.X.toarray()
    sample2_meanGeneExpression = list(np.mean(sample2_geneExpression, axis=0))

    count = 0 

    y = []
    x = []
    
    for i in range(sampleObj.n_vars):
        geneName = geneName_lst[i]

        geneIdx = geneName_df[geneName_df[sampleName] == geneName].index.item()
        geneScore = score_df[sampleName][geneIdx]

        x.append(geneScore)

        sample1 = sample1_meanGeneExpression[i]
        sample2 = sample2_meanGeneExpression[i]

        if sample1 == 0:
            y.append(10)
        elif sample2 == 0:
            y.append(-10)
        elif sample1 == 0 and sample2 == 0:
            y.append(0)
        else:
            sampleDiv = (sample1 / sample2)
            diffGeneExp = math.log2(sampleDiv)
            y.append(diffGeneExp)

    save_list_to_file(x, xFilePath)
    save_list_to_file(y, yFilePath)

#     ticks, labels = yTicks(y)
    
'''    ax.scatter(x, y)
#     ax.set_xlabel(f' {sampleName} DE Score')
    ax.set_xlabel('DE Score')
    ax.set_ylabel('Log Base Differential Gene Expression')
    ax.set_yticks(ticks, labels)
    ax.set_title(f'DE Score vs. Differential Gene Expression - {sample1_name} vs. {sample2_name}')'''

In [ ]:
def savingDiffScatter(sampleComparLst, folderPath, savePath):
    for sampleLst in sampleComparLst: 
        condition = sampleLst[0]

        # Within sample calculation 
        if len(sampleLst) == 2:
            condition = sampleLst[0]
            sampleName = sampleLst[1]
            sample_half1 = f'{sampleName} Half 1'
            sample_half2 = f'{sampleName} Half 2'
            
            sampleFilePaths = search_files(folderPath, sampleName)
            sampleObj = sc.read(sampleFilePaths[0])
            divideby = 'Half'
            fileName = sampleName            
            
            sampleObj.uns['log1p'] = {'base': None}
            
            differentialExpScatter(sampleObj, sample_half1, divideby, f'{savePath}x_{condition}_{sample_half1}', 
                                   f'{savePath}y_{condition}_{sample_half1}')
            differentialExpScatter(sampleObj, sample_half2, divideby,  f'{savePath}x_{condition}_{sample_half2}', 
                                   f'{savePath}y_{condition}_{sample_half2}')
            
        # Between sample calculation 
        else:
            condition = sampleLst[0]
            sample1_name = sampleLst[1]
            sample2_name = sampleLst[2]
            
            sampleFilePaths = search_files(folderPath, f'{sample1_name}_{sample2_name}')
            sampleObj = sc.read(sampleFilePaths[0])
            divideby = 'sample'
            fileName = f'{sample1_name}_{sample2_name}'            
            
            sampleObj.uns['log1p'] = {'base': None}
            
            diffScatterCalc(sampleObj, divideby, savePath, condition)

# Plotting Functions Figure 3

In [ ]:
# PLOTTING FUNCTION: plots the log differential gene expression vs its z-score
def differentialExpScatter(sampleObj, sampleName, divideby, ax):
    sc.tl.rank_genes_groups(sampleObj, divideby, method='wilcoxon')
    
    score_df = pd.DataFrame(sampleObj.uns['rank_genes_groups']['scores'])
    geneName_df = pd.DataFrame(sampleObj.uns['rank_genes_groups']['names'])

    geneName_lst = list(sampleObj.var_names)

    samples = list(np.unique(sampleObj.obs[divideby]))
    sample1_name = samples[0]
    sample2_name = samples[1]
    nameLst = [sample1_name, sample2_name]

    sample1_obj = sampleObj[sampleObj.obs[divideby].isin([sample1_name]),:]
    sample2_obj = sampleObj[sampleObj.obs[divideby].isin([sample2_name]),:]

    sample1_geneExpression = sample1_obj.X.toarray()
    sample1_meanGeneExpression = list(np.mean(sample1_geneExpression, axis=0))

    sample2_geneExpression = sample2_obj.X.toarray()
    sample2_meanGeneExpression = list(np.mean(sample2_geneExpression, axis=0))

    count = 0 

    y = []
    x = []
    
    for i in range(sampleObj.n_vars):
        geneName = geneName_lst[i]

        geneIdx = geneName_df[geneName_df[sampleName] == geneName].index.item()
        geneScore = score_df[sampleName][geneIdx]

        x.append(geneScore)

        sample1 = sample1_meanGeneExpression[i]
        sample2 = sample2_meanGeneExpression[i]

        if sample1 == 0:
            y.append(10)
        elif sample2 == 0:
            y.append(-10)
        elif sample1 == 0 and sample2 == 0:
            y.append(0)
        else:
            sampleDiv = (sample1 / sample2)
            diffGeneExp = math.log2(sampleDiv)
            y.append(diffGeneExp)
    
    ticks, labels = yTicks(y)
    
    plotTitle_fontSize = 15
    
    ax.scatter(x, y, s=7)
    ax.set_xlabel(f' {sampleName} DE Score')
#     ax.set_xlabel('DE Score')
    ax.set_ylabel('Log Base Differential Gene Expression')
    ax.set_yticks(ticks, labels)
    ax.set_title(f'DE Score vs. Differential Gene Expression - {sample1_name} vs. {sample2_name}', 
                 fontsize=plotTitle_fontSize)

In [ ]:
def plottingFig3(sampleComparisonLst, obj_filePath, savePath):
    
    uniqueNum = 0 
    for sampleLst in sampleComparisonLst:
        uniqueNum += 1
        condition = sampleLst[0]
        
        if len(sampleLst) == 2:
            sampleName = sampleLst[1]
            sample1_name = f'{sampleName} Half 1'
            sample2_name = f'{sampleName} Half 2'
            sampleObj_filePath = search_files(obj_filePath, sampleName)
            sampleObj = sc.read(sampleObj_filePath[0])
            sampleObj.uns['log1p'] = {'base': None}
            
            title = f'{sampleName} Half 1 vs. Half 2'
            saveName = f'{sampleName}'
            divideby = 'Half'
            
        else:
            sample1_name = sampleLst[1]
            sample2_name = sampleLst[2]
            sampleObj_filePath = search_files(obj_filePath, f'{sample1_name}_{sample2_name}')
            sampleObj = sc.read(sampleObj_filePath[0])
            sampleObj.uns['log1p'] = {'base': None}
            
            title = f'{sample1_name} vs. Sample {sample2_name}'
            saveName = f'{sample1_name}_{sample2_name}'
            divideby = 'sample'

        # Create a figure with non-grid subplots
        fig = plt.figure(figsize=(30,30), dpi=150, num=uniqueNum)

        # Define the size and position of each subplot using subplot2grid
        row = 4
        col = 8

        ax1 = plt.subplot2grid((row, col), (0, 1), colspan=2) # Clustered UMAP
        ax2 = plt.subplot2grid((row, col), (0, 5), colspan=2) # Clustered UMAP - divided by sample halves

        ax3 = plt.subplot2grid((row, col), (1, 0), colspan=2) # Scatterplot Sample1
        ax5 = plt.subplot2grid((row, col), (1, 4), colspan=2) # Scatterplot Sample2

        ax15 = plt.subplot2grid((row, col), (1, 2), colspan=2) # DE Graph Sample1 
        ax16 = plt.subplot2grid((row, col), (1, 6), colspan=2) # DE Graph Sample2 
        
        ax7 = plt.subplot2grid((row, col), (2, 0), colspan=2) # Sample 1 Top 4 Ranked Genes
        ax8 = plt.subplot2grid((row, col), (2, 2), colspan=2) 
        ax9 = plt.subplot2grid((row, col), (2, 4), colspan=2) 
        ax10 = plt.subplot2grid((row, col), (2, 6), colspan=2) 

        ax11 = plt.subplot2grid((row, col), (3, 0), colspan=2) # Sample 2 Top 4 Ranked Genes
        ax12 = plt.subplot2grid((row, col), (3, 2), colspan=2) 
        ax13 = plt.subplot2grid((row, col), (3, 4), colspan=2)
        ax14 = plt.subplot2grid((row, col), (3, 6), colspan=2) 

        # Plot on each subplot
        # All plot properties
        fontSize = 20

        # Clustered UMAP 
        sc.pl.umap(sampleObj, color = 'leiden_0.3', legend_loc='on data', show = False, ax=ax1)
        ax1.set_title('Clustered UMAP', fontsize=fontSize)

        # Clustered UMAP - Divided by samples
        sc.pl.umap(sampleObj, color = divideby, legend_loc='right margin', show = False, ax=ax2)
        ax2.set_title('Clustered UMAP - Labeled by Sample', fontsize=fontSize)

        # Scatterplot Sample1
        differentialExpScatter(sampleObj, sample1_name, divideby, ax3)

        # DE Graph Sample1
        deGraph(sampleObj, sample1_name, sample2_name, 15, ax15)
        
        # Sample 1 Top 4 Ranked Gene List 
        sample1_topRankedLst = list(sc.get.rank_genes_groups_df(sampleObj, [sample1_name])['names'])
        sample1_top4RankedLst = sample1_topRankedLst[:4]

        # Scatterplot Sample2
        differentialExpScatter(sampleObj, sample2_name, divideby, ax5)
        
        # DE Graph Sample2
        deGraph(sampleObj, sample2_name, sample1_name, 15, ax16)
        
        # Sample 2 Top 4 Ranked Gene List 
        sample2_topRankedLst = list(sc.get.rank_genes_groups_df(sampleObj, [sample2_name])['names'])
        sample2_top4RankedLst = sample2_topRankedLst[:4]

        # Sample 1 Top 4 Ranked Genes
        sc.pl.umap(sampleObj, color = sample1_top4RankedLst[0], vmin=0, vmax='p95', show = False, ax=ax7)
        ax7.set_title(f'{sample1_top4RankedLst[0]} Expression', fontsize=fontSize)

        sc.pl.umap(sampleObj, color = sample1_top4RankedLst[1], vmin=0, vmax='p95', show = False, ax=ax8)
        ax8.set_title(f'{sample1_top4RankedLst[1]} Expression', fontsize=fontSize)

        sc.pl.umap(sampleObj, color = sample1_top4RankedLst[2], vmin=0, vmax='p95', show = False, ax=ax9)
        ax9.set_title(f'{sample1_top4RankedLst[2]} Expression', fontsize=fontSize)

        sc.pl.umap(sampleObj, color = sample1_top4RankedLst[3], vmin=0, vmax='p95', show = False, ax=ax10)
        ax10.set_title(f'{sample1_top4RankedLst[3]} Expression', fontsize=fontSize)

        # Sample 2 Top 4 Ranked Genes
        sc.pl.umap(sampleObj, color = sample2_top4RankedLst[0], vmin=0, vmax='p95', show = False, ax=ax11)
        ax11.set_title(f'{sample2_top4RankedLst[0]} Expression', fontsize=fontSize)

        sc.pl.umap(sampleObj, color = sample2_top4RankedLst[1], vmin=0, vmax='p95', show = False, ax=ax12)
        ax12.set_title(f'{sample2_top4RankedLst[1]} Expression', fontsize=fontSize)

        sc.pl.umap(sampleObj, color = sample2_top4RankedLst[2], vmin=0, vmax='p95', show = False, ax=ax13)
        ax13.set_title(f'{sample2_top4RankedLst[2]} Expression', fontsize=fontSize)

        sc.pl.umap(sampleObj, color = sample2_top4RankedLst[3], vmin=0, vmax='p95', show = False, ax=ax14)
        ax14.set_title(f'{sample2_top4RankedLst[3]} Expression', fontsize=fontSize)


        # Adjust the spacing between subplots
        plt.subplots_adjust(hspace=0.4, wspace=0.4)

        fig.suptitle(f'{condition} - Sample {title} (Samples Clustered Together Using All Genes)', fontsize=30,  fontweight='bold', 
                         y=1, x=0.6)

        # Saving figure
        plt.savefig(os.path.join(savePath, f'{condition}_{saveName}.png'), bbox_inches = 'tight')